In [ ]:
#載入github上的json檔
import pandas as pd
df = pd.read_json('raw.json')
keep_dataframe = df[['jobData','applyData','job_id','cust_id']]

In [ ]:
#應徵分析前的x
import datetime
today = datetime.date.today()
import string
import re

dead_row = [] #要刪掉row
complete_data = []
id_ = 0
job_and_corp = [] #放公司名稱,職缺名稱,公司編號,職缺編號

for unit_data in range(len(keep_dataframe['jobData'])):
    row_data = []
    row_data_percent = []
    row_job_and_corp = []

    if len(keep_dataframe['jobData'][unit_data]) < 17:
        dead_row.append(unit_data)
        continue
    
    row_data.append(id_)
    row_job_and_corp.append(id_)
    
    #職缺名稱
    row_job_and_corp.append(keep_dataframe['jobData'][unit_data]['header']['jobName'])
    #職缺編號
    row_job_and_corp.append(keep_dataframe['job_id'][unit_data])
    
    #更新日期
    day = datetime.datetime.strptime(keep_dataframe['jobData'][unit_data]['header']['appearDate'], '%Y/%m/%d').date()
    row_data.append(abs(today-day).days)
    
    #公司名稱
    row_job_and_corp.append(keep_dataframe['jobData'][unit_data]['header']['custName'])
    #公司編號
    row_job_and_corp.append(keep_dataframe['cust_id'][unit_data])
    
    row_data.append(len(keep_dataframe['jobData'][unit_data]['environmentPic']['environmentPic'])) #環境照片數目
    row_data.append(keep_dataframe['jobData'][unit_data]['condition']['edu']) #學歷要求
    row_data.append(keep_dataframe['jobData'][unit_data]['condition']['major']) #科系要求
    
    #語言要求
    lan_req = []
    for lang in range(len(keep_dataframe['jobData'][unit_data]['condition']["language"])):
        lan_req.append(keep_dataframe['jobData'][unit_data]['condition']["language"][lang]['language'])
    row_data.append(lan_req)
    
    #擅長工具
    specialty = []
    for spe in range(len(keep_dataframe['jobData'][unit_data]['condition']["specialty"])):
        specialty.append(keep_dataframe['jobData'][unit_data]['condition']["specialty"][spe]['description'])
    row_data.append(specialty)
    
    #工作技能
    skill = []
    for sk in range(len(keep_dataframe['jobData'][unit_data]['condition']["skill"])):
        skill.append(keep_dataframe['jobData'][unit_data]['condition']["skill"][sk]['description'])
    row_data.append(skill)
    
    #證照
    certificate = []
    for cer in range(len(keep_dataframe['jobData'][unit_data]['condition']["certificate"])):
        certificate.append(keep_dataframe['jobData'][unit_data]['condition']["certificate"][cer]['name'])
    row_data.append(certificate)
    
    #駕照要求
    if len(keep_dataframe['jobData'][unit_data]['condition']["driverLicense"]) != 0:
        row_data.append(1) 
    else:
        row_data.append(0)
    
    #其他條件（字數）
    other_ = keep_dataframe['jobData'][unit_data]['condition']["other"].replace("\n", "")
    res_other = re.findall(u'([\u4e00-\u9fff0-9a-zA-Z]|(?<=[0-9])[^\u4e00-\u9fff0-9a-zA-Z]+(?=[0-9]))',other_)
    row_data.append(len(res_other))
    
    #福利(文字長度)
    wel_ = ' '.join(keep_dataframe['jobData'][unit_data]['welfare']['tag']) + keep_dataframe['jobData'][unit_data]['welfare']['welfare']
    wel_ = wel_.replace("\n", "")
    res_wel = re.findall(u'([\u4e00-\u9fff0-9a-zA-Z]|(?<=[0-9])[^\u4e00-\u9fff0-9a-zA-Z]+(?=[0-9]))',wel_)
    row_data.append(len(res_wel))
    
    #工作內容（文字長度）
    content = keep_dataframe['jobData'][unit_data]['jobDetail']['jobDescription'].replace("\n", "")
    res = re.findall(u'([\u4e00-\u9fff0-9a-zA-Z]|(?<=[0-9])[^\u4e00-\u9fff0-9a-zA-Z]+(?=[0-9]))',content)
    row_data.append(len(res))
    
    #職務類別
    category = []
    for cat in range(len(keep_dataframe['jobData'][unit_data]['jobDetail']['jobCategory'])):
        category.append(keep_dataframe['jobData'][unit_data]['jobDetail']['jobCategory'][cat]['description'])
    row_data.append(category) 
    
    
    row_data.append(keep_dataframe['jobData'][unit_data]['jobDetail']['addressArea']) #工作地點
    
    #管理責任
    if keep_dataframe['jobData'][unit_data]['jobDetail']['addressArea'] == '不需負擔管理責任':
        row_data.append(0)
    else:
        row_data.append(1)
        
    #出差外派
    if keep_dataframe['jobData'][unit_data]['jobDetail']['businessTrip'] == '無需出差外派':
        row_data.append(0)
    else:
        row_data.append(1)
        
    #上班時段（是否輪班）
    if '需輪班' in keep_dataframe['jobData'][unit_data]['jobDetail']['workPeriod']:
        row_data.append(1)
    else:
        row_data.append(0)
    
    row_data.append(keep_dataframe['jobData'][unit_data]['jobDetail']['vacationPolicy']) #休假制度
    row_data.append(keep_dataframe['jobData'][unit_data]['jobDetail']['startWorkingDay']) #可上班日
    row_data.append(keep_dataframe['jobData'][unit_data]['industry']) #產業類別
    
    id_ += 1
    complete_data.append(row_data)
    job_and_corp.append(row_job_and_corp)

In [ ]:
column_name_df = ['id','appearDate','enviornmentPic','edu','major','language','specialty','skill','certificate',
                 'driverLicense','other','welfare','jobDescription','jobCategory','adressArea','manageResp','businessTrip','workPeriod',
                 'vacationPolicy','startWorkingDay','industry']
job_corp_df = ['id','jobName','job_id','custName','cust_id']
df = pd.DataFrame(complete_data,columns=column_name_df)
name_df = pd.DataFrame(job_and_corp,columns=job_corp_df)

In [ ]:
#名稱檔
name_df.to_csv('x_names.csv')

In [ ]:
need_onehot = ['edu','major','language','specialty','skill','certificate','jobCategory','adressArea','vacationPolicy','startWorkingDay','industry']
df_long = df.explode('major')
one_hot = pd.get_dummies(df_long,columns=['edu','major'])
one_hot = one_hot.groupby('id').max().reset_index()

In [ ]:
one_hot = one_hot.explode('language')
one_hot = pd.get_dummies(one_hot,columns=['language'])
one_hot = one_hot.groupby('id').max().reset_index()

In [ ]:
one_hot = one_hot.explode('specialty')
one_hot = pd.get_dummies(one_hot,columns=['specialty'])
one_hot = one_hot.groupby('id').max().reset_index()

In [ ]:
one_hot = one_hot.explode('skill')
one_hot = pd.get_dummies(one_hot,columns=['skill'])
one_hot = one_hot.groupby('id').max().reset_index()

In [ ]:
one_hot = one_hot.explode('certificate')
one_hot = pd.get_dummies(one_hot,columns=['certificate'])
one_hot = one_hot.groupby('id').max().reset_index()

In [ ]:
one_hot = one_hot.explode('jobCategory')
one_hot = pd.get_dummies(one_hot,columns=['jobCategory'])
one_hot = one_hot.groupby('id').max().reset_index()

In [ ]:
one_hot = one_hot.explode('adressArea')
one_hot = pd.get_dummies(one_hot,columns=['adressArea'])
one_hot = one_hot.groupby('id').max().reset_index()

In [ ]:
one_hot = one_hot.explode('vacationPolicy')
one_hot = pd.get_dummies(one_hot,columns=['vacationPolicy'])
one_hot = one_hot.groupby('id').max().reset_index()

In [ ]:
one_hot = one_hot.explode('startWorkingDay')
one_hot = pd.get_dummies(one_hot,columns=['startWorkingDay'])
one_hot = one_hot.groupby('id').max().reset_index()

In [ ]:
one_hot = one_hot.explode('industry')
one_hot = pd.get_dummies(one_hot,columns=['industry'])
one_hot = one_hot.groupby('id').max().reset_index()

In [ ]:
#出現少於20次的類別(edu)
#'edu_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('edu'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)
#加'edu_其他'
edu_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        edu_else.append(1)
    else:
        edu_else.append(0)
#加'edu_其他'，drop edu_ columns
one_hot['edu_其他'] = edu_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
#出現少於20次的類別(major)
#'major_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('major'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)

#'major_其他'
major_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        major_else.append(1)
    else:
        major_else.append(0)
#加'major_其他'，drop edu_ columns
one_hot['major_其他'] = major_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
#出現少於20次的類別(language)
#'language_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('language'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)

#'language_其他'
language_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        language_else.append(1)
    else:
        language_else.append(0)
#加'language_其他'，drop edu_ columns
one_hot['language_其他'] = language_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
#出現少於20次的類別(specialty)
#'specialty_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('specialty'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)

#'specialty_其他'
specialty_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        specialty_else.append(1)
    else:
        specialty_else.append(0)
#加'specialty_其他'，drop specialty_ columns
one_hot['specialty_其他'] = specialty_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
#出現少於20次的類別(skill)
#'skill_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('skill'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)

#'skill_其他'
skill_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        skill_else.append(1)
    else:
        skill_else.append(0)
#加'skill_其他'，drop skill_ columns
one_hot['skill_其他'] = skill_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
#出現少於20次的類別(certificate)
#'certificate_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('certificate'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)

#'certificate_其他'
certificate_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        certificate_else.append(1)
    else:
        certificate_else.append(0)
#加'certificate_其他'，drop skill_ columns
one_hot['certificate_其他'] = certificate_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
#出現少於20次的類別(jobCategory)
#'jobCategory_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('jobCategory'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)

#'jobCategory_其他'
jobCategory_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        jobCategory_else.append(1)
    else:
        jobCategory_else.append(0)
#加'jobCategory_其他'，drop jobCategory_ columns
one_hot['jobCategory_其他'] = jobCategory_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
#出現少於20次的類別(adressArea)
#'adressArea_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('adressArea'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)

#'adressArea_其他'
adressArea_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        adressArea_else.append(1)
    else:
        adressArea_else.append(0)
#加'adressArea_其他'，drop adressArea_ columns
one_hot['adressArea_其他'] = adressArea_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
#出現少於20次的類別(vacationPolicy)
#'vacationPolicy_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('vacationPolicy'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)

#'vacationPolicy_其他'
vacationPolicy_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        vacationPolicy_else.append(1)
    else:
        vacationPolicy_else.append(0)
#加'vacationPolicy_其他'，drop vacationPolicy_ columns
one_hot['vacationPolicy_其他'] = vacationPolicy_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
#出現少於20次的類別(startWorkingDay)
#'startWorkingDay_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('startWorkingDay'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)

#'startWorkingDay_其他'
startWorkingDay_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        startWorkingDay_else.append(1)
    else:
        startWorkingDay_else.append(0)
#加'startWorkingDay_其他'，drop startWorkingDay_ columns
one_hot['startWorkingDay_其他'] = startWorkingDay_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
#出現少於20次的類別(industry)
#'industry_其他'要改為1的row
delete_col = []
record_row_all = []
for s in list(one_hot.columns):
    if s.startswith('industry'):
        if one_hot[s].sum() < 20:
            delete_col.append(s)
            record_row = [index for index, value in enumerate(list(one_hot[s])) if value == 1]
            for re in record_row:
                if re in record_row_all:
                    pass
                else:
                    record_row_all.append(re)

#'industry_其他'
industry_else = []
num_row = len(keep_dataframe['jobData'])-len(dead_row)
for node in range(num_row):
    if node in record_row_all:
        industry_else.append(1)
    else:
        industry_else.append(0)
#加'industry_其他'，drop industry_ columns
one_hot['industry_其他'] = industry_else
one_hot = one_hot.drop(delete_col, axis=1)

In [ ]:
one_hot = one_hot.drop(['id'], axis=1)

In [ ]:
#x變數（不包含應徵分析）
one_hot

In [ ]:
one_hot.to_csv('without_apply_x.csv')

In [ ]:
#合起來
all_in = pd.concat([one_hot, one_hot_apply], axis = 1)

In [ ]:
#變數x資料標準化
from sklearn.preprocessing import StandardScaler
all_in[['other', 'welfare', 'jobDescription']] = StandardScaler().fit_transform(all_in[['other', 'welfare', 'jobDescription']])